# MDP Query with DuckDB

A demonstrator notebook for DuckDB operations against Parquet files.  The goal is to develop and performance test queries in SQL against Parquet.


<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/mdpDuckDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>



In [ ]:
%%capture
!pip install -q minio

In [2]:
import duckdb
import pandas as pd
import numpy as np
import os
import spacy
import tqdm
from minio import Minio

# Using DuckDB with local Parquet from MDP

In [3]:
lcon = duckdb.connect()
lcon.execute("CREATE TABLE data AS SELECT  row_number() OVER () AS idx, * FROM read_parquet('./mdpProduct.parquet')")  # load from url

In [4]:
r = lcon.execute("DESCRIBE SELECT * FROM data").fetchdf()
print(r)

              column_name column_type null  key default extra
0                     idx      BIGINT  YES  NaN     NaN   NaN
1                      id     VARCHAR  YES  NaN     NaN   NaN
2                keywords     VARCHAR  YES  NaN     NaN   NaN
3                    type     VARCHAR  YES  NaN     NaN   NaN
4                 geotype     VARCHAR  YES  NaN     NaN   NaN
5                geompred     VARCHAR  YES  NaN     NaN   NaN
6                    geom     VARCHAR  YES  NaN     NaN   NaN
7        temporalCoverage     VARCHAR  YES  NaN     NaN   NaN
8           datePublished     VARCHAR  YES  NaN     NaN   NaN
9                 license     VARCHAR  YES  NaN     NaN   NaN
10                creator     VARCHAR  YES  NaN     NaN   NaN
11  includedInDataCatalog     VARCHAR  YES  NaN     NaN   NaN
12           distribution     VARCHAR  YES  NaN     NaN   NaN
13              publisher     VARCHAR  YES  NaN     NaN   NaN
14           filteredgeom     VARCHAR  YES  NaN     NaN   NaN
15      

In [12]:
r = lcon.execute(" SELECT DISTINCT  type, COUNT(*) AS count FROM data GROUP BY   type").fetchdf()
print(r)

             type  count
0  schema:Dataset   2015


# Using DuckDB with S3 based Parquet files 

In [2]:
def publicurls(client, bucket, prefix):
    urls = []
    objects = client.list_objects(bucket, prefix=prefix, recursive=True)
    for obj in objects:
        result = client.stat_object(bucket, obj.object_name)

        if result.size > 0:  #  how to tell if an objet   obj.is_public  ?????
            url = client.presigned_get_object(bucket, obj.object_name)
            # print(f"Public URL for object: {url}")
            urls.append(url)

    return urls

client = Minio("ossapi.oceaninfohub.org:80",  secure=False) # Create client with anonymous access.
urls = publicurls(client, "public", "assets")
for u in urls:
  print(u)

http://ossapi.oceaninfohub.org/public/assets/OIHGraph_25032023.parquet
http://ossapi.oceaninfohub.org/public/assets/africaioc.parquet
http://ossapi.oceaninfohub.org/public/assets/cioos.parquet
http://ossapi.oceaninfohub.org/public/assets/combined.parquet
http://ossapi.oceaninfohub.org/public/assets/edmerp.parquet
http://ossapi.oceaninfohub.org/public/assets/edmo.parquet
http://ossapi.oceaninfohub.org/public/assets/emodnet.parquet
http://ossapi.oceaninfohub.org/public/assets/gridcounts.gpkg
http://ossapi.oceaninfohub.org/public/assets/inanodc.parquet
http://ossapi.oceaninfohub.org/public/assets/invemardocuments.parquet
http://ossapi.oceaninfohub.org/public/assets/invemarexperts.parquet
http://ossapi.oceaninfohub.org/public/assets/invemarinstitutions.parquet
http://ossapi.oceaninfohub.org/public/assets/invemartraining.parquet
http://ossapi.oceaninfohub.org/public/assets/invemarvessels.parquet
http://ossapi.oceaninfohub.org/public/assets/marinetraining.parquet
http://ossapi.oceaninfohub.o

In [3]:
## load the combined graph
urlCombined = "http://ossapi.oceaninfohub.org/public/assets/combined.parquet"
urlCIOOS = "http://ossapi.oceaninfohub.org/public/assets/cioos.parquet"

duckdb.install_extension("httpfs")

# Instantiate the DuckDB connection
con = duckdb.connect()
con.execute("CREATE TABLE data AS SELECT  row_number() OVER () AS idx, * FROM read_parquet('{}')".format(urlCombined))  # load from url

con2 = duckdb.connect()
con2.execute("CREATE TABLE data AS SELECT  row_number() OVER () AS idx, * FROM read_parquet('{}')".format(urlCIOOS))  # load from url


# con.execute("CREATE TABLE my_table AS SELECT * FROM read_parquet('/content/drive/MyDrive/Data/combined.parquet')") # load from local parquet
# con.execute("CREATE TABLE data AS SELECT row_number() OVER () AS idx, * FROM '/content/drive/MyDrive/Data/combined.parquet';")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
r = con.execute("DESCRIBE SELECT * FROM data").fetchdf()
print(r)

         column_name column_type null  key default extra
0                idx      BIGINT  YES  NaN     NaN   NaN
1                  s     VARCHAR  YES  NaN     NaN   NaN
2               type     VARCHAR  YES  NaN     NaN   NaN
3               name     VARCHAR  YES  NaN     NaN   NaN
4           keywords     VARCHAR  YES  NaN     NaN   NaN
5                url     VARCHAR  YES  NaN     NaN   NaN
6               desc     VARCHAR  YES  NaN     NaN   NaN
7            provder     VARCHAR  YES  NaN     NaN   NaN
8  __index_level_0__      BIGINT  YES  NaN     NaN   NaN


In [6]:
r = con2.execute("DESCRIBE SELECT * FROM data").fetchdf()
print(r)

              column_name column_type null  key default extra
0                     idx      BIGINT  YES  NaN     NaN   NaN
1                      id     VARCHAR  YES  NaN     NaN   NaN
2                keywords     VARCHAR  YES  NaN     NaN   NaN
3                    type     VARCHAR  YES  NaN     NaN   NaN
4                    name     VARCHAR  YES  NaN     NaN   NaN
5             description     VARCHAR  YES  NaN     NaN   NaN
6                     url     VARCHAR  YES  NaN     NaN   NaN
7                 geotype     VARCHAR  YES  NaN     NaN   NaN
8                geompred     VARCHAR  YES  NaN     NaN   NaN
9                    geom     VARCHAR  YES  NaN     NaN   NaN
10       temporalCoverage     VARCHAR  YES  NaN     NaN   NaN
11          datePublished     VARCHAR  YES  NaN     NaN   NaN
12                license     VARCHAR  YES  NaN     NaN   NaN
13                creator     VARCHAR  YES  NaN     NaN   NaN
14  includedInDataCatalog     VARCHAR  YES  NaN     NaN   NaN
15      

In [6]:
# Now you can execute SQL queries on the Parquet file as if it was a regular table
r = con.execute("SELECT DISTINCT provder FROM data").fetchdf()

print(r)

                provder
0          oceanexperts
1                   pdh
2              aquadocs
3                 cioos
4                edmerp
5                  edmo
6               emodnet
7               inanodc
8      invemardocuments
9        invemarexperts
10  invemarinstitutions
11      invemartraining
12       invemarvessels
13       marinetraining
14                 obis
15                 obps
16            africaioc


In [7]:
# Now you can execute SQL queries on the Parquet file as if it was a regular table
r = con.execute(" SELECT DISTINCT provder, type, COUNT(*) AS count FROM data GROUP BY provder, type").fetchdf()


print(r)

NameError: name 'con' is not defined

In [8]:
# Now you can execute SQL queries on the Parquet file as if it was a regular table
r = con.execute(" SELECT DISTINCT provder, type, ANY_VALUE(s),  COUNT(*) AS count FROM data GROUP BY provder, type  order by count desc").fetchdf()

print(r)

                provder                      type  \
0              aquadocs       schmea:CreativeWork   
1              aquadocs             schmea:Person   
2                 cioos       schemawrong:Dataset   
3          oceanexperts             schmea:Person   
4                   pdh       schemawrong:Dataset   
5          oceanexperts              schmea:Event   
6                  obis            schmea:Dataset   
7      invemardocuments       schmea:CreativeWork   
8              aquadocs       schmea:Organization   
9                edmerp       schmea:Organization   
10     invemardocuments             schmea:Person   
11                 obps       schmea:CreativeWork   
12                 obps             schmea:Person   
13         oceanexperts       schmea:Organization   
14                 edmo       schmea:Organization   
15     invemardocuments       schmea:Organization   
16                  pdh  schemawrong:Organization   
17               edmerp    schmea:ResearchProj